In [9]:
api_key = os.getenv("GOOGLE_API_KEY", "").strip()

if not api_key:
    print("GOOGLE_API_KEY não encontrada.")
else:
    try:
        genai.configure(api_key=api_key)

        modelos = []
        for m in genai.list_models():
            metodos = getattr(m, "supported_generation_methods", []) or []
            modelos.append({
                "name": getattr(m, "name", ""),
                "display_name": getattr(m, "display_name", ""),
                "description": getattr(m, "description", ""),
                "supported_generation_methods": ", ".join(metodos),
            })

        df_modelos = pd.DataFrame(modelos).sort_values("name").reset_index(drop=True)
        print(df_modelos.to_string(index=False))
    except Exception as e:
        print(f"[Erro ao listar modelos] {type(e).__name__}: {e}")

                                                name                                       display_name                                                                                                                                                                                         description                                            supported_generation_methods
                                          models/aqa Model that performs Attributed Question Answering.                                                                   Model trained to return answers to questions that are grounded in provided sources, along with estimating answerable probability.                                                          generateAnswer
            models/deep-research-pro-preview-12-2025            Deep Research Pro Preview (Dec-12-2025)                                                                                                                                          Preview release

In [10]:
import os
import json
import requests
import pandas as pd
import google.generativeai as genai
from getpass import getpass

# 1) Exemplo de tratamento de exceções (ValueError, TypeError, ZeroDivisionError + geral)
def operacao_riscada(a, b, arquivo_log="log_temp.txt"):
    arquivo = None
    try:
        arquivo = open(arquivo_log, "w", encoding="utf-8")

        if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
            raise TypeError("Parâmetros devem ser numéricos.")
        if b == 0:
            raise ZeroDivisionError("Divisão por zero não é permitida.")
        if a < 0:
            raise ValueError("Valor inválido: 'a' não pode ser negativo neste exemplo.")

        resultado = a / b
        arquivo.write(f"Resultado: {resultado}\n")
        return resultado

    except ValueError as e:
        print(f"[ValueError] Verifique os valores informados: {e}")
    except TypeError as e:
        print(f"[TypeError] Tipos inválidos. Dica: use int/float. Detalhe: {e}")
    except ZeroDivisionError as e:
        print(f"[ZeroDivisionError] {e}")
    except Exception as e:
        print(f"[Erro inesperado] {type(e).__name__}: {e}")
    finally:
        if arquivo:
            arquivo.close()
            print("Arquivo de log fechado no bloco finally.")


# Testes rápidos
_ = operacao_riscada(10, 2)
_ = operacao_riscada("10", 2)
_ = operacao_riscada(-1, 2)
_ = operacao_riscada(10, 0)


# 2) Filtrar DataFrame por análises negativas e unir resenhas
df_neg = df[df["analise"].str.lower().eq("negativa")].copy()
resenhas_negativas = df_neg["resenha"].dropna().astype(str).tolist()
texto_unificado = " || ".join(resenhas_negativas)

print("Total resenhas negativas:", len(resenhas_negativas))
print("Texto unificado:", texto_unificado)


# 3) Cliente LLM (Gemini 2.5 Flash) + chamada com parâmetros
def chamar_gemini(prompt, model_name="gemini-2.5-flash", temperature=0.2, top_p=0.9, max_tokens=300):
    try:
        chave = (api_key or os.getenv("GOOGLE_API_KEY", "")).strip()
        if not chave:
            raise RuntimeError("GOOGLE_API_KEY não encontrada.")

        genai.configure(api_key=chave)
        model = genai.GenerativeModel(model_name=model_name)
        resposta = model.generate_content(
            prompt,
            generation_config={
                "temperature": temperature,
                "top_p": top_p,
                "max_output_tokens": max_tokens,
            },
        )

        texto = getattr(resposta, "text", "")
        if texto:
            return texto.strip()

        partes = []
        for cand in getattr(resposta, "candidates", []) or []:
            content = getattr(cand, "content", None)
            for part in getattr(content, "parts", []) or []:
                t = getattr(part, "text", "")
                if t:
                    partes.append(t)
        return "\n".join(partes).strip()

    except Exception as e:
        print(f"[Erro LLM Google] {type(e).__name__}: {e}")
        return ""


# Prompt inicial
prompt_categorias = f"""
Analise as resenhas negativas abaixo e extraia categorias principais de problema.
Resenhas: {texto_unificado}
Retorne as categorias separadas por vírgula.
"""

resposta_categorias = chamar_gemini(prompt_categorias)
print("Categorias (bruto):", resposta_categorias)


# Prompt refinado: 1 palavra por categoria
prompt_categorias_1_palavra = f"""
Com base nas resenhas negativas abaixo, retorne apenas categorias de problema.
Regras:
- uma palavra por categoria
- separadas por vírgula
- sem explicações

Resenhas: {texto_unificado}
"""

resposta_1_palavra = chamar_gemini(prompt_categorias_1_palavra)
categorias_lista = [c.strip() for c in resposta_1_palavra.split(",") if c.strip()]
print("Categorias (lista):", categorias_lista)


# 4) Extração de JSON por resenha (texto -> dict)
def extrair_json_por_resenha(resenha):
    prompt_json = f"""
Classifique a resenha e devolva APENAS JSON válido neste formato:
{{"categoria":"...", "sentimento":"negativo", "gravidade":"baixa|media|alta"}}

Resenha: {resenha}
"""
    texto = chamar_gemini(prompt_json, max_tokens=120).strip()
    texto = texto.replace("```json", "").replace("```", "").strip()

    try:
        return json.loads(texto)
    except json.JSONDecodeError:
        inicio = texto.find("{")
        fim = texto.rfind("}")
        if inicio != -1 and fim != -1 and fim > inicio:
            try:
                return json.loads(texto[inicio:fim + 1])
            except Exception:
                pass
        return {
            "categoria": "desconhecida",
            "sentimento": "negativo",
            "gravidade": "media",
            "raw": texto,
        }


json_dicts = [extrair_json_por_resenha(r) for r in resenhas_negativas]
print("JSONs convertidos para dict:", json_dicts)


# 5) Modelo local (LM Studio/Ollama)
def chamar_ollama(prompt, model="gemma3:1b"):
    # Exemplo de seleção/baixa:
    #   ollama pull gemma3:1b
    #   ollama serve
    try:
        url = "http://localhost:11434/api/generate"
        payload = {"model": model, "prompt": prompt, "stream": False}
        r = requests.post(url, json=payload, timeout=60)
        r.raise_for_status()
        return r.json().get("response", "").strip()
    except Exception as e:
        print(f"[Erro LLM local] {type(e).__name__}: {e}")
        return ""


resposta_local = chamar_ollama("Resuma em uma frase: " + texto_unificado)
print("Resposta local:", resposta_local)


# 6) Processamento em lote + união do resultado final
def processar_lote_resenhas(df_entrada, col_resenha="resenha", col_analise="analise", col_nota="nota"):
    df_n = df_entrada[df_entrada[col_analise].str.lower().eq("negativa")].copy()
    lista = df_n[col_resenha].dropna().astype(str).tolist()
    contagem_notas = df_n[col_nota].value_counts(dropna=False).sort_index().to_dict()
    resultados = [extrair_json_por_resenha(x) for x in lista]
    return {
        "total_negativas": len(lista),
        "contagem_notas": contagem_notas,
        "resultados_json": resultados,
    }


def unir_resultados_texto(resultado_lote):
    linhas = [
        f"Total negativas: {resultado_lote['total_negativas']}",
        f"Contagem de notas: {resultado_lote['contagem_notas']}",
        "Categorias detectadas:",
    ]
    for i, item in enumerate(resultado_lote["resultados_json"], start=1):
        linhas.append(f"{i}. {item.get('categoria', 'desconhecida')} (gravidade: {item.get('gravidade', 'n/a')})")
    return "\n".join(linhas)


resultado_lote = processar_lote_resenhas(df)
texto_final = unir_resultados_texto(resultado_lote)

print("\n=== RELATÓRIO FINAL ===")
print(texto_final)


# 7) Ambiente virtual + IDE (referência)
# python -m venv .venv
# source .venv/bin/activate      # Linux/Mac
# .venv\Scripts\activate         # Windows
# pip install pandas requests google-generativeai
# Use VS Code/PyCharm para executar o notebook/script.

Arquivo de log fechado no bloco finally.
[TypeError] Tipos inválidos. Dica: use int/float. Detalhe: Parâmetros devem ser numéricos.
Arquivo de log fechado no bloco finally.
[ValueError] Verifique os valores informados: Valor inválido: 'a' não pode ser negativo neste exemplo.
Arquivo de log fechado no bloco finally.
[ZeroDivisionError] Divisão por zero não é permitida.
Arquivo de log fechado no bloco finally.
Total resenhas negativas: 2
Texto unificado: Entrega atrasou e veio danificado. || Atendimento ruim e suporte não resolve.
Categorias (bruto): Entrega, Qualidade do Produto, Atendimento ao Cliente
Categorias (lista): ['Entrega', 'Dano', 'Atendimento', 'Su']
JSONs convertidos para dict: [{'categoria': 'desconhecida', 'sentimento': 'negativo', 'gravidade': 'media', 'raw': '{'}, {'categoria': 'desconhecida', 'sentimento': 'negativo', 'gravidade': 'media', 'raw': '{'}]
[Erro LLM local] ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /ap